In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import os

# pandas options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# selenium imports
from selenium import webdriver

In [3]:
# final
url = 'https://garasi.id/pencarian/mobil-bekas/semua?q=min_year%3D2015&province_id%5B0%5D=5&province_id%5B1%5D=10&province_id%5B2%5D=11&province_id%5B3%5D=12&province_id%5B4%5D=17&province_id%5B5%5D=18&min_year=2015&min_price=&is_featured=&sort=price-asc'

In [4]:
# load url in chrome webdriver
driver = webdriver.Chrome()
driver.get(url)

# scroll down to get to list of brands
driver.execute_script("window.scrollTo(0, 300)")
time.sleep(1)

In [10]:
# source: https://www.statista.com/statistics/981088/indonesia-new-vehicle-sale/
# source: https://www.thejakartapost.com/life/2018/01/20/gaikindo-reveals-five-popular-car-brands-in-indonesia-in-2017.html
# top 5 brands in Indonesia
brands = ['Toyota', 'Honda', 'Daihatsu', 'Mitsubishi', 'Suzuki']
models = ['Ertiga']
#models = ['Avanza', 'Kijang Innova', 'Calya', 'Fortuner', 'Agya', 'Rush', 'Yaris',
#          'Brio', 'Mobilio', 'HR-V', 'Jazz', 'BR-V', 'CR-V',
#          'Xenia', 'Sigra', 'Ayla', 'Terios', 'Pajero Sport', 'Xpander', 'Ertiga']

In [11]:
def scrape_url(currmod_idx):
    listing_url = []
    featured = []
    
    # iterate through brands
    for i in range(2,15):
        car_brand = driver.find_element_by_xpath("//*[@id='root']/div[1]/main/div/div/div[3]/div[1]/div[2]/div/form/div[4]/div[3]/div/div/div[2]/div[" + str(i) + "]/div")                                                    

        # if found brand
        if car_brand.text in brands:
            car_brand.click()    
            model_labels = driver.find_elements_by_class_name('InputSelectionRow_Label')

            # iterate through list of models
            for j in range(len(model_labels)):
                if model_labels[j].text == models[currmod_idx]:
                    ########## click filter
                    checkbox = model_labels[j].find_element_by_class_name('InputSelectionRow_Selection')
                    checkbox.click()

                    ########## More scrolling to expand on number of listings
                    SCROLL_PAUSE_TIME = 1

                    for k in range(5):
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                        time.sleep(SCROLL_PAUSE_TIME)        

                    ####### Collect listing url here
                    links = driver.find_elements_by_class_name('link-none')

                    # iterate through list of listings
                    for k in range(2,len(links)):
                        itm = links[k]
                        # add listing url
                        listing_url.append(itm.get_property("href"))
                        # check if listing is featured
                        feat = itm.find_elements_by_tag_name("img")
                        f = False
                        for l in feat:
                            if l.get_property("title") == 'mobil pilihan':
                                f = True
                        featured.append(f)

                    ##### scroll to top
                    driver.execute_script("window.scrollTo(0, 0)")    

                    # close filter
                    close = driver.find_element_by_xpath('//*[@id="root"]/div[1]/main/div/div[2]/div[2]/form/div[1]/span[2]')
                    close.click()

                    # scroll down to get to list of brands
                    driver.execute_script("window.scrollTo(0, 300)")
                    time.sleep(1)

                    # click again to close accordion
                    car_brand = driver.find_element_by_xpath("//*[@id='root']/div[1]/main/div/div/div[3]/div[1]/div[2]/div/form/div[4]/div[3]/div/div/div[2]/div[" + str(i) + "]/div")
                    car_brand.click()
                    return listing_url, featured
                    
    # dummy return for fail
    return -1, -1

In [7]:
all_listing = []
all_featured = []

In [12]:
# scrape url for all models
for i in range(len(models)):
    print('Scraping: ', models[i])
    listing, featured = scrape_url(i)
    all_listing.extend(listing)
    all_featured.extend(featured)

driver.close()

Scraping:  Ertiga


In [13]:
all_listing[-5:]

['https://garasi.id/mobil-bekas/2015-suzuki-ertiga/00pz138',
 'https://garasi.id/mobil-bekas/2017-suzuki-ertiga/00ou972',
 'https://garasi.id/mobil-bekas/2018-suzuki-ertiga/00qe045',
 'https://garasi.id/mobil-bekas/2018-suzuki-ertiga/00qg147',
 'https://garasi.id/mobil-bekas/2015-suzuki-ertiga/00py107']

In [14]:
df = pd.DataFrame()
df['listing_url'] = all_listing
df['featured'] = all_featured

In [15]:
df.head()

,listing_url,featured
0,https://garasi.id/mobil-bekas/2017-toyota-avan...,True
1,https://garasi.id/mobil-bekas/2015-toyota-avan...,True
2,https://garasi.id/mobil-bekas/2015-toyota-avan...,False
3,https://garasi.id/mobil-bekas/2016-toyota-avan...,False
4,https://garasi.id/mobil-bekas/2019-toyota-avan...,False


In [16]:
df.shape # has some duplicates

(3687, 2)

In [17]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [18]:
df.shape

(2610, 2)

In [19]:
df.to_csv('listing_url.csv', index = False) # remove duplicates